In [1]:
!pip install SpeechRecognition
!pip install ffmpeg-python
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg
!pip install PyAudio
!pip install googletrans==4.0.0-rc1
!pip install translate
!pip install gtts
!pip install torch
!pip install TTS


# Install the SpeechRecognition library for speech recognition tasks.
!pip install SpeechRecognition

# Install the ffmpeg-python library for working with audio and video files using FFmpeg.
!pip install ffmpeg-python

# Install necessary system-level dependencies for audio processing.
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg

# Install the PyAudio library for audio input/output in Python.
!pip install PyAudio

# Install the googletrans library for text translation using Google Translate.
!pip install googletrans==4.0.0-rc1

# Install the translate library for text translation between languages.
!pip install translate

# Install the gtts (Google Text-to-Speech) library for text-to-speech conversion.
!pip install gtts

# Install the torch library, which is usually PyTorch for deep learning.
!pip install torch

# Install the TTS library, which may be a custom library for text-to-speech or a specific package.
!pip install TTS

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 13.8 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libasound2-dev is already the newest version (1.2.6.1-1ubuntu1).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 29 not upgraded.
Need to get 188 kB of archives.
After this operation, 927 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudiocpp0 amd64 19.6.0-1.1 [16.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 portaudio19-dev amd64 19.6.0-1.1 [106 kB]
Fetched 188 kB in 2s (100 kB/s)
Selecting previously unselected package libportaudio2:am

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libasound2-dev is already the newest version (1.2.6.1-1ubuntu1).
libportaudio2 is already the newest version (19.6.0-1.1).
libportaudiocpp0 is already the newest version (19.6.0-1.1).
portaudio19-dev is already the newest version (19.6.0-1.1).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [2]:
import pyaudio
import wave
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg
import gc

In [3]:
gc.collect()

44

In [4]:
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});

</script>
"""

def get_audio():
    display(HTML(AUDIO_HTML))
    data = eval_js("data")
    binary = b64decode(data.split(',')[1])

    # Конвертирование аудио в формат WAV
    process = (ffmpeg
               .input('pipe:0')
               .output('pipe:1', format='wav')
               .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
               )
    output, err = process.communicate(input=binary)

    riff_chunk_size = len(output) - 8
    # Break up the chunk size into four bytes, held in b.
    q = riff_chunk_size
    b = []
    for i in range(4):
        q, r = divmod(q, 256)
        b.append(r)

    # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
    riff = output[:4] + bytes(b) + output[8:]

    sr, audio = wav_read(io.BytesIO(riff))

    # Сохранение аудио в файле WAV
    with open("recorded_audio.wav", "wb") as wav_file:
        wav_file.write(riff)

    return audio, sr


In [5]:
gc.collect()

0

# **translator**

In [6]:
# Language used for speech recognition (English, USA)
speech_language = 'ar-EG'

  # Specify the target language for translation (French in this case)
target_language = 'ru'  # You can change this to the desired language code

In [13]:
audio, sr = get_audio()

In [9]:
gc.collect()

0

Here *** playing audio directly in the notebook***

In [14]:
import speech_recognition as sr
from googletrans import Translator
from gtts import gTTS
from IPython.display import Audio, display
import os

def translate_text(original_text, target_language):
    try:
        # Use Google Translate to translate the text to the specified target language
        translator = Translator()
        translated_text = translator.translate(original_text, dest=target_language).text
        return translated_text
    except Exception as e:
        # Handle any exceptions that may occur during translation
        print(f"Translation error: {e}")
        return None

def speak_text(text, language='en'):
    try:
        # Create a gTTS object
        tts = gTTS(text=text, lang=language, slow=False)

        # Save the TTS audio to a file
        tts.save("translated_audio.mp3")

        # Play the TTS audio in the notebook
        display(Audio(filename="translated_audio.mp3", autoplay=True))
    except Exception as e:
        print(f"Text-to-speech error: {e}")

# Path to the audio file
OUTPUT = '/content/recorded_audio.wav'

# Initialize SpeechRecognition recognizer and audio file
recognizer = sr.Recognizer()
audio_file = sr.AudioFile(OUTPUT)

try:
    # Use SpeechRecognition to recognize speech from the audio file
    with audio_file as source:
        audio_data = recognizer.record(source)

    # Perform speech-to-text recognition
    original_text = recognizer.recognize_google(audio_data, language=speech_language)
    print("Original Text:", original_text)

    # Translate the recognized text to the specified target language
    translated_text = translate_text(original_text, target_language)

    if translated_text is not None:
        print(f"Translated Text ({target_language}): {translated_text}")

        # Convert translated text to speech and play in the notebook
        speak_text(translated_text, language=target_language)
    else:
        print("Translation failed.")

except sr.UnknownValueError:
    print("Google Speech Recognition could not understand audio.")
except sr.RequestError as e:
    print(f"Could not request results from Google Speech Recognition service; {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


Original Text: قول اي شيء قول اي حاجه يعني البرنامج ده مش راضي يشتغل ولا راضي يشتغل ليه
Translated Text (ru): Сказать что -нибудь, чтобы сказать какую -либо потребность, я имею в виду, эта программа не удовлетворена, и она не удовлетворена этим


In [15]:
from TTS.api import TTS
# tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2", gpu=True)
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2", gpu=False)

# # generate speech by cloning a voice using default settings
tts.tts_to_file(text = translated_text,
                file_path="output.wav",
                speaker_wav="/content/recorded_audio.wav",
                language="ar")

# # # generate speech by cloning a voice using custom settings
# tts.tts_to_file(text=translated_text,
#                 file_path="output2.wav",
#                 speaker_wav="/content/recorded_audio.wav",
#                 language="ru",
#                 decoder_iterations=30)


 > You must confirm the following:
 | > "I have purchased a commercial license from Coqui: licensing@coqui.ai"
 | > "Otherwise, I agree to the terms of the non-commercial CPML: https://coqui.ai/cpml" - [y/n]
 | | > y
 > Downloading model to /root/.local/share/tts/tts_models--multilingual--multi-dataset--xtts_v2


100%|█████████▉| 1.86G/1.87G [00:27<00:00, 84.1MiB/s]
100%|██████████| 1.87G/1.87G [00:28<00:00, 66.5MiB/s]
100%|██████████| 4.37k/4.37k [00:00<00:00, 8.51kiB/s]
 55%|█████▍    | 198k/361k [00:00<00:00, 451kiB/s] 
100%|██████████| 361k/361k [00:01<00:00, 339kiB/s]
100%|██████████| 32.0/32.0 [00:00<00:00, 34.0iB/s]
 96%|█████████▌| 7.41M/7.75M [00:00<00:00, 74.1MiB/s]

 > Model's license - CPML
 > Check https://coqui.ai/cpml.txt for more info.
 > Using model: xtts


100%|██████████| 7.75M/7.75M [00:19<00:00, 74.1MiB/s]

 > Text splitted to sentences.
['Сказать что -нибудь, чтобы сказать какую -либо потребность, я имею в виду, эта программа не удовлетворена, и она не удовлетворена этим']
 > Processing time: 46.20560431480408
 > Real-time factor: 4.5953019013018235


'output.wav'

In [ ]:
gc.collect()

50474

In [16]:
from IPython.display import Audio

# Assuming the file name is 'recorded_audio.wav'
audio_path = 'recorded_audio.wav'

# Play the audio
Audio(audio_path)

In [17]:
from IPython.display import Audio

# Assuming the file name is 'output.wav'
audio_path = '/content/output.wav'

# Play the audio
Audio(audio_path)